## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-02-12-17 +0000,startribune,Live: Vikings trail the Bears 17-6 in the thir...,https://www.startribune.com/minnesota-vikings-...
1,2025-09-09-02-07-27 +0000,bbc,US lawmakers release Epstein 'birthday book' w...,https://www.bbc.com/news/articles/cvgqnn4ngvdo...
2,2025-09-09-02-05-13 +0000,nypost,Two people killed in fast-moving fire that tor...,https://nypost.com/2025/09/08/us-news/two-peop...
3,2025-09-09-01-55-01 +0000,nypost,New York’s Saw Mill River Parkway shut down af...,https://nypost.com/2025/09/08/us-news/new-york...
4,2025-09-09-01-53-25 +0000,nyt,Firefighters Race to Save a Treasured Sequoia ...,https://www.nytimes.com/2025/09/08/us/sequoia-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,40
25,new,20
203,court,14
18,killed,14
21,fire,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
177,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...,102
74,2025-09-08-22-10-46 +0000,nypost,Inside Curtis Sliwa’s decades-old feud with Tr...,https://nypost.com/2025/09/08/us-news/inside-c...,89
225,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...,89
246,2025-09-08-11-26-15 +0000,cbc,At least 19 killed as police open fire on Nepa...,https://www.cbc.ca/news/world/nepal-deadly-pro...,89
52,2025-09-08-23-07-20 +0000,nypost,Homeless perv busted for sex attack inside NYC...,https://nypost.com/2025/09/08/us-news/homeless...,86


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
177,102,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...
74,89,2025-09-08-22-10-46 +0000,nypost,Inside Curtis Sliwa’s decades-old feud with Tr...,https://nypost.com/2025/09/08/us-news/inside-c...
246,89,2025-09-08-11-26-15 +0000,cbc,At least 19 killed as police open fire on Nepa...,https://www.cbc.ca/news/world/nepal-deadly-pro...
316,49,2025-09-08-03-32-23 +0000,bbc,What it was like inside court as mushroom murd...,https://www.bbc.com/news/videos/c15k4y5nwz7o?a...
225,49,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...
60,40,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
266,39,2025-09-08-09-46-24 +0000,nypost,6 dead in deadly terrorist attack at Jerusalem...,https://nypost.com/2025/09/08/world-news/four-...
29,38,2025-09-09-00-10-07 +0000,nypost,Ukrainian refugee Iryna Zarutska’s father coul...,https://nypost.com/2025/09/08/us-news/ukrainia...
208,36,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...
85,35,2025-09-08-21-41-45 +0000,bbc,Supreme Court lifts limits on LA immigration ...,https://www.bbc.com/news/articles/c784697j2v8o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
